In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import OneHotEncoder
import logging
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Lambda

# Setup logging

In [2]:

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Detect device
def get_device():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                details = tf.config.experimental.get_device_details(gpu)
                device_name = details.get("device_name", "").upper()
                if "NVIDIA" in device_name:
                    logger.info("CUDA GPU detected. Using NVIDIA GPU.")
                    return 'GPU'
                elif "APPLE" in device_name or "METAL" in device_name:
                    logger.info("Metal GPU detected. Using Apple M-series GPU.")
                    return 'GPU'
            logger.info("Unknown GPU type detected. Defaulting to GPU.")
            return 'GPU'
        except Exception as e:
            logger.warning(f"Could not get GPU details: {e}")
            return 'GPU'
    else:
        logger.info("No compatible GPU detected. Using CPU.")
        tf.config.set_visible_devices([], 'GPU')
        return 'CPU'

device_type = get_device()

# Generate fake chess games for training
def generate_chess_game(game_id):
    np.random.seed(game_id)
    white_elo = np.random.randint(300, 3000)
    black_elo = np.random.randint(300, 3000)
    num_moves = np.random.randint(5, 16)
    white_times = [300.0]
    black_times = [300.0]
    for _ in range(num_moves - 1):
        white_times.append(white_times[-1] - np.random.randint(1, 10) + 5)
        black_times.append(black_times[-1] - np.random.randint(1, 10) + 5)
    white_evals = [np.random.randint(-50, 51)]
    black_evals = [-white_evals[0]]
    for _ in range(num_moves - 1):
        white_evals.append(np.clip(white_evals[-1] + np.random.randint(-20, 21), -50, 50))
        black_evals.append(-white_evals[-1])
    white_moves = list(range(0, num_moves * 2, 2))
    black_moves = list(range(1, num_moves * 2, 2))
    missed_castling = np.random.choice([0, 1])
    missed_mate_in_5 = np.random.randint(0, 3)
    game_data = {
        "white": {
            "elo": white_elo,
            "times": white_times,
            "evals": white_evals,
            "moves": white_moves
        },
        "black": {
            "elo": black_elo,
            "times": black_times,
            "evals": black_evals,
            "moves": black_moves
        },
        "game_features": {
            "missed_castling": missed_castling,
            "missed_mate_in_5": missed_mate_in_5
        }
    }
    return game_data
game_data_list = [generate_chess_game(i) for i in range(100)]
print(game_data_list)

INFO:__main__:Metal GPU detected. Using Apple M-series GPU.


[{'white': {'elo': 2907, 'times': [300.0, 301.0, 298.0, 297.0, 297.0], 'evals': [20, 24, 28, 20, 1], 'moves': [0, 2, 4, 6, 8]}, 'black': {'elo': 1953, 'times': [300.0, 301.0, 302.0, 304.0, 301.0], 'evals': [-20, -24, -28, -20, -1], 'moves': [1, 3, 5, 7, 9]}, 'game_features': {'missed_castling': 0, 'missed_mate_in_5': 2}}, {'white': {'elo': 1361, 'times': [300.0, 299.0, 303.0, 300.0, 302.0, 301.0, 301.0, 301.0, 298.0, 295.0, 293.0, 291.0, 295.0], 'evals': [-42, -38, -45, -50, -40, -50, -50, -50, -49, -50, -50, -46, -50], 'moves': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]}, 'black': {'elo': 535, 'times': [300.0, 304.0, 307.0, 305.0, 305.0, 307.0, 309.0, 306.0, 309.0, 313.0, 310.0, 313.0, 316.0], 'evals': [42, 38, 45, 50, 40, 50, 50, 50, 49, 50, 50, 46, 50], 'moves': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25]}, 'game_features': {'missed_castling': 0, 'missed_mate_in_5': 0}}, {'white': {'elo': 2875, 'times': [300.0, 298.0, 294.0, 296.0, 295.0, 295.0, 292.0, 290.0, 291.0, 289.0, 

In [ ]:

# Parse a game to input features and (optionally) target ELO

# --- 1. Parse Game Function with Scalar Features ---
def parse_game(game_json, max_seq_length=128, include_target=True):
    if isinstance(game_json, str):
        game_data = json.loads(game_json)
    else:
        game_data = game_json

    white_data = game_data['white']
    black_data = game_data['black']
    sequence = []
    max_moves = max(len(white_data['moves']), len(black_data['moves']))
    sum_time_white = sum(white_data['times'])
    sum_time_black = sum(black_data['times'])
    for i in range(max_moves):
        if i < len(white_data['moves']):
            time = white_data['times'][i] / sum_time_white
            eval_score = white_data['evals'][i] / 100.0
            move_num = white_data['moves'][i] / 40.0
            sequence.append([time, eval_score, move_num, 1.0])
        if i < len(black_data['moves']):
            time = black_data['times'][i] / sum_time_black
            eval_score = black_data['evals'][i] / 100.0
            move_num = black_data['moves'][i] / 40.0
            sequence.append([time, eval_score, move_num, 0.0])
    sequence = sequence[:max_seq_length] + [[0,0,0,0]] * (max_seq_length - len(sequence))
    sequence = np.array(sequence, dtype=np.float32)

    # Scalar game-level features
    features = game_data["game_features"]
    missed_castling = float(features.get("missed_castling", 0))
    missed_mate_in_5 = float(features.get("missed_mate_in_5", 0))
    scalar_features = np.array([missed_castling, missed_mate_in_5], dtype=np.float32)

    if include_target:
        white_elo = white_data['elo'] / 3000.0
        black_elo = black_data['elo'] / 3000.0
        target = np.array([white_elo, black_elo], dtype=np.float32)
        return (sequence, scalar_features), target
    else:
        return sequence, scalar_features


In [30]:


# --- 2. Dataset Generator ---
def generator(data_list, max_seq_length=128):
    for game in data_list:
        yield parse_game(game, max_seq_length)

def create_tf_dataset(game_data_list, batch_size=4):
    output_signature = (
        (
            tf.TensorSpec(shape=(128, 4), dtype=tf.float32),  # move sequence
            tf.TensorSpec(shape=(2,), dtype=tf.float32)       # scalar features
        ),
        tf.TensorSpec(shape=(2,), dtype=tf.float32)            # ELO targets
    ) 
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(game_data_list),
        output_signature=output_signature
    )
    return dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# --- 3. Transformer Block ---
def transformer_block(x, num_heads, ff_dim, dropout_rate=0.1):
    # Multi-head self-attention
    attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x)
    attention_output = layers.Dropout(dropout_rate)(attention_output)
    attention_output = layers.LayerNormalization(epsilon=1e-6)(x + attention_output)

    # Feed-forward
    ff_output = layers.Dense(ff_dim, activation='relu')(attention_output)
    ff_output = layers.Dense(x.shape[-1])(ff_output)
    ff_output = layers.Dropout(dropout_rate)(ff_output)
    output = layers.LayerNormalization(epsilon=1e-6)(attention_output + ff_output)

    return output

# --- 4. Chess Model with Scalar Features ---
def build_chess_model(input_shape=(128, 4), d_model=256, num_heads=4, ff_dim=1024, num_layers=6, meta_shape=(2,)):
    seq_inputs = layers.Input(shape=input_shape)
    meta_inputs = layers.Input(shape=meta_shape)

    # Project input to d_model dimensions
    x = layers.Dense(d_model)(seq_inputs)

    # Positional Embedding
    positions = tf.range(start=0, limit=input_shape[0], delta=1)
    pos_embedding = layers.Embedding(input_dim=input_shape[0], output_dim=d_model)(positions)
    x = x + pos_embedding

    # Transformer Layers
    for _ in range(num_layers):
        x = transformer_block(x, num_heads, ff_dim)

    x = layers.GlobalAveragePooling1D()(x)

    # Concatenate game-level features
    x = layers.Concatenate()([x, meta_inputs])
    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(2, activation='linear')(x)

    return models.Model(inputs=[seq_inputs, meta_inputs], outputs=outputs)


In [ ]:

# --- 5. Training Function ---
def run_elo_prediction(game_data_list):
    split_idx = int(0.9 * len(game_data_list))
    train_data = game_data_list[:split_idx]
    val_data = game_data_list[split_idx:]

    train_dataset = create_tf_dataset(train_data)
    val_dataset = create_tf_dataset(val_data)

    model = build_chess_model()
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=2
    )

    model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=50,
        callbacks=[early_stop]
    )
    return model

# --- 6. Prediction Function ---
def predict_elo(model, game_json):
    sequence, scalar_features = parse_game(game_json, include_target=False)
    sequence = np.expand_dims(sequence, axis=0)
    scalar_features = np.expand_dims(scalar_features, axis=0)
    pred = model.predict([sequence, scalar_features])[0]
    white_elo = pred[0] * 3000
    black_elo = pred[1] * 3000
    return white_elo, black_elo

# Run training and prediction


In [31]:
model = run_elo_prediction(game_data_list)

# Save model with proper extension
model.save("model/chess_elo_prediction_model.keras")
logger.info("Model training complete and saved!")

# Predict a new game (without Elo in input)
new_game = '{"white": {"times": [300.0, 298.0, 295.0, 290.0, 285.0], "evals": [5, 8, 12, 15, 20], "moves": [0, 2, 4, 6, 8]}, "black": {"times": [300.0, 297.0, 292.0, 287.0, 283.0], "evals": [-5, -8, -10, -12, -18], "moves": [1, 3, 5, 7, 9]}, "game_features": {"missed_castling": 0, "missed_mate_in_5": 1}}'
white_elo, black_elo = predict_elo(model, new_game)
logger.info(f"Predicted Elo Ratings for New Game:")
logger.info(f"White Player: {white_elo:.2f}")
logger.info(f"Black Player: {black_elo:.2f}")


Epoch 1/50
    225/Unknown 51s 168ms/step - loss: 4.7716 - mae: 1.2631

2025-04-18 12:40:21.727361: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9581292381100507758
2025-04-18 12:40:21.727811: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14216116361957441646
2025-04-18 12:40:21.727819: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 13406493532066497592
2025-04-18 12:40:21.727823: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 6680782140473532313
2025-04-18 12:40:21.727826: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 13233452408811685954
2025-04-18 12:40:21.727828: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 18404516722414580225
2025-04-18 12:40:21.727836: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv

225/225 ━━━━━━━━━━━━━━━━━━━━ 53s 177ms/step - loss: 4.7590 - mae: 1.2615 - val_loss: 0.1977 - val_mae: 0.3708
Epoch 2/50


2025-04-18 12:40:23.823230: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 7687695610991014138
2025-04-18 12:40:23.823248: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4647536944056451318
2025-04-18 12:40:23.823252: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 12259773844594655473
2025-04-18 12:40:23.823261: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16833472092879274626
2025-04-18 12:40:23.823263: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 83001825764008461
2025-04-18 12:40:23.823265: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 17982211449731293504
2025-04-18 12:40:23.823267: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv it

225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 0.8640 - mae: 0.7354

...

2025-04-18 12:47:27.319320: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14216116361957441646
2025-04-18 12:47:27.319339: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16841989017620381208
2025-04-18 12:47:27.319345: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16495867710500832333
2025-04-18 12:47:27.319369: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 3037507670347040607
2025-04-18 12:47:27.319386: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4164564355405514995
2025-04-18 12:47:27.319405: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9581292381100507758
2025-04-18 12:47:27.319420: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv 

225/225 ━━━━━━━━━━━━━━━━━━━━ 39s 174ms/step - loss: 0.1486 - mae: 0.3112 - val_loss: 0.0756 - val_mae: 0.2371
Epoch 12: early stopping
Restoring model weights from the end of the best epoch: 9.


2025-04-18 12:47:28.550074: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 7687695610991014138
2025-04-18 12:47:28.550090: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4647536944056451318
2025-04-18 12:47:28.550094: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 12259773844594655473
2025-04-18 12:47:28.550104: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 83001825764008461
2025-04-18 12:47:28.550106: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16833472092879274626
2025-04-18 12:47:28.550111: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4027799597492796805
2025-04-18 12:47:28.550113: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv ite

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 679ms/step


INFO:__main__:Predicted Elo Ratings for New Game:
INFO:__main__:White Player: 1496.74
INFO:__main__:Black Player: 1538.69


In [25]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 128, 4)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 128, 256)  │      1,280 │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_39 (Add)        │ (None, 128, 256)  │          0 │ dense_43[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 256)  │  1,051,904 │ add_39[0][0],     │
│ (MultiHeadAttentio… │                   │            │ add_39[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_57          │ (None, 128, 256)  │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_40 (Add)        │ (None, 128, 256)  │          0 │ add_39[0][0],     │
│                     │                   │            │ dropout_57[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 256)  │        512 │ add_40[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 128, 1024) │    263,168 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_45 (Dense)    │ (None, 128, 256)  │    262,400 │ dense_44[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_58          │ (None, 128, 256)  │          0 │ dense_45[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_41 (Add)        │ (None, 128, 256)  │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_58[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 256)  │        512 │ add_41[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 256)  │  1,051,904 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_60          │ (None, 128, 256)  │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_42 (Add)        │ (None, 128, 256)  │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_60[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 256)  │        512 │ add_42[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_46 (Dense)    │ (None, 128, 1024) │    263,168 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_47 (Dense)    │ (None, 128, 256)  │    262,400 │ dense_46[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_61          │ (None, 128, 256)  │          0 │ dense_47[0][0]  

 Total params: 28,517,000 (108.78 MB)

 Trainable params: 9,505,666 (36.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,011,334 (72.52 MB)